A ideia desse material é de capturar as informações de todos os arquivos do site do [Jornal Oficial de Mossoró](http://jom.prefeiturademossoro.com.br).

A versão do Python esperada é no mínimo a 3.8.

As dependencias que precisaremos para fazer todo o trabalho são:
- httpx: responsavel por lidar com as requisições
- parse: nos permite fazer buscas a partir de padrões
- pyquery: nos permite fazer buscar no estilo jquery

In [291]:
pip install -r requirements.txt

Note: you may need to restart the kernel to use updated packages.


Importando o que vamos precisar usar

In [126]:
import httpx
from pyquery import PyQuery
from parse import search

from typing import List

In [244]:
jom_url_base = "http://jom.prefeiturademossoro.com.br"
jom_edicoes_url_base = f"{jom_url_base}/todas-as-edicoes-2/"

def acessar_edicoes_jom_por_pagina(pagina: int) -> str:
    jom_url_edicoes_paginas = f"{jom_edicoes_url_base}?_page={pagina}"
    jom_edicoes_response = httpx.get(jom_url_edicoes_paginas)
    
    return jom_edicoes_response.text

In [162]:
classe_para_busca_dos_documentos = ".pt-cv-ifield"

def pegar_elementos_de_arquivos_na_pagina(html_pagina: str) -> List:
    html_para_busca = PyQuery(html_pagina)
    lista_elementos_arquivos = html_para_busca(classe_para_busca_dos_documentos)
    
    return [item for item in lista_elementos_arquivos.items()]

In [245]:
def pegar_os_links_dos_documentos_por_elementos(elementos: List) -> List:
    return [
        href for elem in elementos
        if (href := elem("a:first").attr("href"))
    ]

In [246]:
def pegar_todos_os_links_de_documentos() -> List:
    todos_os_links = []
    numero_da_pagina = 1
    while ((resposta := acessar_edicoes_jom_por_pagina(numero_da_pagina)) and
           (elementos := pegar_elementos_de_arquivos_na_pagina(resposta)) and 
           (links := pegar_os_links_dos_documentos_por_elementos(elementos))):
        todos_os_links += links
        numero_da_pagina += 1

    return todos_os_links

In [247]:
def acessar_pagina_jom_do_arquivo_por_url(url_da_pagina_do_arquivo: str) -> str:
    jom_arquivo_response = httpx.get(url_da_pagina_do_arquivo)
    
    return jom_arquivo_response.text

In [283]:
classe_para_busca_das_informacos = "div.entry-content:first"

def pegar_informacoes_do_arquivos_na_pagina(html_pagina: str) -> List:
    html_para_busca = PyQuery(html_pagina)
    lista_elementos_arquivos = html_para_busca(classe_para_busca_das_informacos)
    
    # Temos que iterar um generator para que possamos pegar o unico elemento
    # que queremos da pagina
    elemento = [item for item in lista_elementos_arquivos.items()][0]
    elemento_em_texto = elemento.text()
    
    return {
        "link_download": elemento("iframe:first").attr("src"),
        # Algumas paginas possuem a formatação de data errada:
        # <dia> de <mes> de <ano>: padrão
        # <dia> de <mes> <ano>: http://jom.prefeiturademossoro.com.br/2016/05/18/jom-n-356a/
        # <dia> <mes> de <ano>: http://jom.prefeiturademossoro.com.br/2014/02/14/jom-n-240/
        "data": (search("Data: {dia:d} de {mes:S} de {ano:d}", elemento_em_texto) or 
                 search("Data: {dia:d} de {mes:S} {ano:d}", elemento_em_texto) or
                 search("Data: {dia:d} {mes:S} de {ano:d}", elemento_em_texto)).named,
        "ano": search("Ano: {:S}", elemento_em_texto)[0],
        # Algumas páginas possuem nomes diferentes edição/número:
        # Número: padrão
        # Edição nº: http://jom.prefeiturademossoro.com.br/2008/05/09/jom-n-o-07a/
        "numero": (search("Número: {:w}", elemento_em_texto) or
                   search("Edição nº: {:w}", elemento_em_texto))[0],
        # Algumas paginas possuem o numero de paginas errado:
        # N.º de páginas: padrão
        # N.º Páginas: http://jom.prefeiturademossoro.com.br/2008/05/09/jom-n-o-07a/
        # Páginas: http://jom.prefeiturademossoro.com.br/2008/01/30/jom-n-o-01-ano-1-quarta-feira-30-de-janeiro-de-2008/
        # "XX": http://jom.prefeiturademossoro.com.br/2019/11/22/jom-n-o-537/
        "qnt_paginas": (search("N.º de páginas: {:d}", elemento_em_texto) or
                        search("N.º Páginas: {:d}", elemento_em_texto) or
                        search("Páginas: {:d}", elemento_em_texto) or
                        [None])[0],
    }

In [250]:
lista_url_paginas_documentos = pegar_todos_os_links_de_documentos()

In [289]:
informacoes = []

for url_documento in lista_url_paginas_documentos:
    pagina_html = acessar_pagina_jom_do_arquivo_por_url(url_documento)
    informacoes.append(
        pegar_informacoes_do_arquivos_na_pagina(pagina_html)
    )

print(len(informacoes))

1394
